In [2]:
import os
import pandas as pd
import random
#import matplotlib.pyplot as plt

In [3]:
path2 = "/Users/Richard/PycharmProjects/NeuroLab_3.10/WordList/EmoIma/ESM1_categorised.csv"
Emo = pd.read_csv(path2, sep=",")
#Emo.head(50)

## Initial categorisation to filter lists 

In [4]:
#Define Conditions

#set threshold
lower_threshold_emo = -33
upper_threshold_emo = 33

#set threshold Ima
lower_threshold_ima = 33
upper_threshold_ima = 66

# Define emo conditions
emo_mid_condition = (Emo["EM_Y_MEAN"] < upper_threshold_emo) & (Emo["EM_O_MEAN"] < upper_threshold_emo) & (Emo["EM_Y_MEAN"] > lower_threshold_emo) & (Emo["EM_O_MEAN"] > lower_threshold_emo)

# Define ima conditions
ima_high_condition = (Emo["IM_Y_MEAN"] > upper_threshold_ima) & (Emo["IM_O_MEAN"] > upper_threshold_ima)
ima_mid_condition = (Emo["IM_Y_MEAN"] < upper_threshold_ima) & (Emo["IM_O_MEAN"] < upper_threshold_ima) & (Emo["IM_Y_MEAN"] > lower_threshold_ima) & (Emo["IM_O_MEAN"] > lower_threshold_ima)


In [5]:
#Add condition column to emo df
# Initialize new columns with default values
Emo['emo_condition'] = 'None'
Emo['ima_condition'] = 'None'

# Apply conditions and set values
Emo.loc[emo_mid_condition, 'emo_condition'] = 'emo_mid'
Emo.loc[ima_high_condition, 'ima_condition'] = 'ima_high'
Emo.loc[ima_mid_condition, 'ima_condition'] = 'ima_mid'

In [6]:
#Test results
#Emo[['WORD', "EM_Y_MEAN", "EM_O_MEAN", "IM_Y_MEAN", "IM_O_MEAN", 'emo_condition', 'ima_condition']].head(50)

## Utils 

In [7]:
def write_to_excel(file_name, df):
    output_path = "/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/Output"
    file_path = os.path.join(output_path, file_name)
    df.to_excel(file_path, index=False)
    return f"{file_name} writen to {file_path}"


In [8]:
# Function to generate word pairs with different first letters with checks

#TODO: Add function that uses the gen_word pair function and creates a list for all the conditions that you want to have

#condition_pairs = [("emo_mid", "ima_mid"), ("emo_mid", "ima_high")]

# Function to generate word pairs with different first letters from two different DataFrames
def generate_word_pairs(df1, list_of_condition_pairs, num_pairs_per_condition, seed=42):
    
    # Set seed for reproducibility
    random.seed(seed)
    
    #initalize lists
    word_pairs = []
    used_words = []
    
    for cond_pair in list_of_condition_pairs:
        
        #create dfs based on specified conditions
        emo_words = df1[df1['emo_condition']== cond_pair[0]][['WORD', 'emo_condition']]
        emo_words_tuples = list(zip(emo_words['WORD'], emo_words['emo_condition']))
    
        ima_words = df1[df1['ima_condition']== cond_pair[1]][['WORD', 'ima_condition']]
        ima_words_tuples = list(zip(ima_words['WORD'], ima_words['ima_condition']))
    
        pairs_counter = 0
        attempts_counter = 0
        max_attempts = (len(ima_words_tuples)+len(emo_words_tuples))/2
        
        while pairs_counter < num_pairs_per_condition and attempts_counter < max_attempts:
            attempts_counter += 1
            
            word1 = random.choice(emo_words_tuples)  # Draw randomly from emo_words
            word2 = random.choice(ima_words_tuples)  # Draw randomly from ima_words
            
            # Check if both words are not already used in any word pair
            if (word1[0] not in used_words and 
                word2[0] not in used_words and 
                word1[0][0] != word2[0][0]):
                word_pairs.append((word1, word2))
                used_words.append(word1[0])
                used_words.append(word2[0])
                pairs_counter += 1 
        
        if attempts_counter >= max_attempts:
            print(f"Warning: Maximum attempts reached for condition pair {cond_pair}. Only {pairs_counter} pairs were generated.")
    
    # Unzip the list of tuples into two separate lists
    list1, list2 = zip(*word_pairs)
    
    # Further unzip each of these lists if necessary
    word1, condition1 = zip(*list1)
    word2, condition2 = zip(*list2)

    # Create a DataFrame from these lists
    df = pd.DataFrame({
        'Word1': word1,
        'Emo_condition': condition1,
        'Word2': word2,
        'Ima_condition': condition2
    })     
        
    return df

#list_test = generate_word_pairs(Emo,condition_pairs,18)
#print(generate_word_pairs(emo_high,"WORD", ima_high, "WORD", 10))

#list_test

## Generate Nature List

In [10]:
# Filter list to get only nature Words
Emo_words_nature = Emo[Emo['Kategorie'] == 'nature_based_non_human']
Emo_short_words_nature = Emo[(Emo['length_group'] == 'short_word') & (Emo['Kategorie'] == 'nature_based_non_human')]
Emo_long_words_nature = Emo[(Emo['length_group'] == 'long_word') & (Emo['Kategorie'] == 'nature_based_non_human')]

In [11]:
#Set Condition Pair to be generated
condition_pairs = [("emo_mid", "ima_high")]

#Generate wordpairs for short words and for long words
cond_short_nature = generate_word_pairs(Emo_short_words_nature,condition_pairs,18)
cond_long_nature = generate_word_pairs(Emo_long_words_nature,condition_pairs,18)

In [ ]:
short_nature_new = generate_word_pairs(Emo_short_words_nature,condition_pairs,18)

In [15]:
condition_pairs_primacy_recency = [("None", "None")]
nature_dummy = generate_word_pairs(Emo_long_words_nature,condition_pairs_primacy_recency,2, -13)
nature_dummy

,Word1,Emo_condition,Word2,Ima_condition
0,Maikäfer,None,Kardinal,None
1,Karpfen,None,Vanille,None


In [48]:
cond_new = [("emo_mid", "ima_high")]
generate_word_pairs(Emo_long_words_nature,cond_new,1, 70)
#nature_words_new = pd.concat([cond_short_nature, cond_new, nature_dummy], ignore_index=True)
#nature_words_new

,Word1,Emo_condition,Word2,Ima_condition
0,Habicht,emo_mid,Kolibri,ima_high


In [12]:
#Condition pair none so it's not a word within the emo/ ima categories
condition_pairs_primacy_recency = [("None", "None")]

#create dummy words and split in 2 dataframes
dummy_words = generate_word_pairs(Emo, condition_pairs_primacy_recency, 6, 2)
primacy_words = dummy_words.iloc[:3]
recency_words = dummy_words.iloc[3:]

#Concat all individual dfs
nature_words = pd.concat([primacy_words,cond_short_nature, cond_long_nature, recency_words], ignore_index=True)
nature_words

,Word1,Emo_condition,Word2,Ima_condition
0,Diebstahl,None,Misserfolg,None
1,Gewalt,None,Krieg,None
2,Kompott,None,Vorschlag,None
3,Spore,emo_mid,Bulle,ima_high
4,Araber,emo_mid,Vampir,ima_high
5,Hafer,emo_mid,Gras,ima_high
6,Gaul,emo_mid,Ei,ima_high
7,Wolf,emo_mid,Buche,ima_high
8,Quitte,emo_mid,Bock,ima_high
9,Rohr,emo_mid,Larve,ima_high


In [13]:
nature_words["WORD_length1"] = nature_words["Word1"].apply(len)
nature_words["WORD_length2"] = nature_words["Word2"].apply(len)

mean_length = Emo["WORD_length"].mean()

def length_categorization(word):
    if len(word) < mean_length:
        return "short_word"
    elif len(word) > mean_length:
        return "long_word"
    
nature_words["length_group_emo"] = nature_words["Word1"].apply(length_categorization)
nature_words["length_group_ima"] = nature_words["Word2"].apply(length_categorization)


#Reorganize columns
cols = nature_words.columns.tolist()
cols = ['Word1', 'WORD_length1', 'Emo_condition','length_group_emo','Word2',"WORD_length2",'Ima_condition', "length_group_ima"]


nature_words= nature_words[cols]

#final result
nature_words

,Word1,WORD_length1,Emo_condition,length_group_emo,Word2,WORD_length2,Ima_condition,length_group_ima
0,Diebstahl,9,None,long_word,Misserfolg,10,None,long_word
1,Gewalt,6,None,short_word,Krieg,5,None,short_word
2,Kompott,7,None,long_word,Vorschlag,9,None,long_word
3,Spore,5,emo_mid,short_word,Bulle,5,ima_high,short_word
4,Araber,6,emo_mid,short_word,Vampir,6,ima_high,short_word
5,Hafer,5,emo_mid,short_word,Gras,4,ima_high,short_word
6,Gaul,4,emo_mid,short_word,Ei,2,ima_high,short_word
7,Wolf,4,emo_mid,short_word,Buche,5,ima_high,short_word
8,Quitte,6,emo_mid,short_word,Bock,4,ima_high,short_word
9,Rohr,4,emo_mid,short_word,Larve,5,ima_high,short_word


In [41]:
write_to_excel('nature_words_new.xlsx',nature_words_new)

'nature_words_new.xlsx writen to /Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/Output/nature_words_new.xlsx'

## Generate list for non-nature words

In [22]:
Emo.head()

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_O_MEAN,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O,Kategorie,WORD_length,length_group,emo_condition,ima_condition
0,Aal,11,10,12,10,90.55,93.40,97.33,90.50,12.30,...,36.16,36.58,41.77,0.0,0.0,nature_based_non_human,3,short_word,None,ima_high
1,Aas,10,10,13,11,73.10,57.50,80.69,59.64,20.44,...,-62.74,24.86,32.97,0.0,0.0,NaN,3,short_word,None,None
2,Abend,10,11,10,12,53.80,52.27,59.90,74.08,28.96,...,50.47,27.44,26.39,0.0,0.0,NaN,5,short_word,None,None
3,Abendbrot,9,11,13,10,64.89,62.64,84.85,82.50,18.72,...,59.00,23.44,29.92,0.0,0.0,NaN,9,long_word,None,None
4,Abfall,8,10,13,9,73.38,77.80,67.46,60.89,28.84,...,-45.63,23.08,28.66,0.0,0.0,NaN,6,short_word,None,None


In [27]:
# Filter list to get only nature Words
Emo_words_non_nature = Emo[Emo['Kategorie'].isna()]
Emo_short_words_non_nature = Emo_words_non_nature[(Emo_words_non_nature['length_group'] == 'short_word')]
Emo_long_words_non_nature = Emo_words_non_nature[(Emo_words_non_nature['length_group'] == 'long_word')]

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_O_MEAN,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O,Kategorie,WORD_length,length_group,emo_condition,ima_condition
3,Abendbrot,9,11,13,10,64.89,62.64,84.85,82.50,18.72,...,59.00,23.44,29.92,0.0,0.0,NaN,9,long_word,None,None
5,Abgrund,11,9,11,11,72.27,61.33,72.64,68.64,23.04,...,-58.79,41.59,30.18,0.0,0.0,NaN,7,long_word,None,ima_high
7,Abnahme,11,10,13,10,23.64,20.60,27.38,44.50,22.20,...,6.95,30.32,23.59,0.0,0.0,NaN,7,long_word,emo_mid,None
8,Abrechnung,10,9,13,10,47.10,28.00,58.08,52.50,30.52,...,2.26,37.37,20.25,0.0,0.0,NaN,10,long_word,None,ima_mid
10,Absicht,10,9,12,11,14.40,12.00,18.92,42.18,17.12,...,7.58,37.46,9.12,0.0,0.0,NaN,7,long_word,emo_mid,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2583,Zuwachs,10,11,12,11,25.30,24.73,51.92,43.09,23.61,...,38.05,28.24,30.82,0.0,0.0,NaN,7,long_word,None,None
2584,Zweifel,11,9,13,10,29.55,20.33,34.23,33.90,21.64,...,-24.26,26.11,27.41,0.0,0.0,NaN,7,long_word,emo_mid,None
2587,Zwieback,8,11,12,11,80.00,92.00,95.00,93.27,22.70,...,42.42,26.83,28.69,0.0,0.0,NaN,8,long_word,None,ima_high
2589,Zwielicht,9,11,10,12,45.11,30.27,57.60,52.33,28.84,...,-30.30,28.49,34.31,0.0,0.0,NaN,9,long_word,emo_mid,ima_mid


In [50]:
cond_non_new = [("emo_mid", "ima_high")]
generate_word_pairs(Emo_long_words_non_nature,cond_non_new,3, 70)

,Word1,Emo_condition,Word2,Ima_condition
0,Werbung,emo_mid,Einfahrt,ima_high
1,Gewitter,emo_mid,Sprudel,ima_high
2,Malkasten,emo_mid,Ohrring,ima_high


In [28]:

#Set Condition Pair to be generated
condition_pairs = [("emo_mid", "ima_high")]

#Generate wordpairs for short words and for long words
cond_short_non_nature = generate_word_pairs(Emo_short_words_non_nature, condition_pairs, 18)
cond_long_non_nature = generate_word_pairs(Emo_long_words_non_nature, condition_pairs, 18)

#Concat all individual dfs (use same primacy/ recency words as in nature word list)
non_nature_words = pd.concat([primacy_words, cond_short_non_nature, cond_long_non_nature, recency_words], ignore_index=True)

non_nature_words["WORD_length1"] = non_nature_words["Word1"].apply(len)
non_nature_words["WORD_length2"] = non_nature_words["Word2"].apply(len)

mean_length = Emo["WORD_length"].mean()

#apply length categorization
non_nature_words["length_group_emo"] = non_nature_words["Word1"].apply(length_categorization)
non_nature_words["length_group_ima"] = non_nature_words["Word2"].apply(length_categorization)

#Reorganize columns
cols = non_nature_words.columns.tolist()
cols = ['Word1', 'WORD_length1', 'Emo_condition', 'length_group_emo', 'Word2', "WORD_length2", 'Ima_condition',
        "length_group_ima"]

non_nature_words = non_nature_words[cols]

#final result
non_nature_words

,Word1,WORD_length1,Emo_condition,length_group_emo,Word2,WORD_length2,Ima_condition,length_group_ima
0,Diebstahl,9,None,long_word,Misserfolg,10,None,long_word
1,Gewalt,6,None,short_word,Krieg,5,None,short_word
2,Kompott,7,None,long_word,Vorschlag,9,None,long_word
3,Wodka,5,emo_mid,short_word,Dreck,5,ima_high,short_word
4,Bagger,6,emo_mid,short_word,Kappe,5,ima_high,short_word
5,Kelch,5,emo_mid,short_word,Haus,4,ima_high,short_word
6,Gang,4,emo_mid,short_word,Deckel,6,ima_high,short_word
7,Stufe,5,emo_mid,short_word,Büste,5,ima_high,short_word
8,Tuch,4,emo_mid,short_word,Napf,4,ima_high,short_word
9,Eis,3,emo_mid,short_word,Handy,5,ima_high,short_word


In [31]:
write_to_excel('non_nature_words.xlsx',non_nature_words)

'non_nature_words.xlsx writen to /Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/Output/non_nature_words.xlsx'

In [2]:
def write_to_excel(file_name, df):
    output_path = "/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/Output"
    file_path = os.path.join(output_path, file_name)
    df.to_excel(file_path, index=False)
    return f"{file_name} writen to {file_path}"


## Some checks to test if there are errors in the list

In [37]:
#Check to see if a word from Emo Word list is also in Ima Word list
nature_words_new["Word1"].isin(nature_words_new["Word2"])

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
Name: Word1, dtype: bool

In [40]:
#check if a word appears more then once

# Count the occurrences of each word in the 'words' column
word_counts_emo = nature_words_new['Word1'].value_counts()
word_counts_ima = nature_words_new['Word2'].value_counts()

# Filter words that appear more than once
repeated_words_emo = word_counts_emo[word_counts_emo > 1]
repeated_words_ima = word_counts_ima[word_counts_ima > 1]

print("Words that appear more than once:")
print(repeated_words_emo)

print("Words that appear more than once:")
print(repeated_words_ima)

Words that appear more than once:
Series([], Name: count, dtype: int64)
Words that appear more than once:
Series([], Name: count, dtype: int64)
